In [1]:
# Randomly pick the values of S, X, R in different time

import numpy as np
n = 6000
S = np.random.randint(0.5*n, 0.8*n, size=60)
S_array = np.sort(S)[::-1]
remaining = n*np.ones(60) - S_array
R_array = np.zeros(60)

for r in range(60):
    R_array[r] = np.random.randint(R_array[r-1]+1, n-S_array[r])

X_array = remaining - R_array

In [2]:
# Calculate the values of beta and gamma

beta_array = -n * np.diff(S_array) / (S_array[:-1] * X_array[:-1])
gamma_array = np.diff(R_array) / X_array[:-1]

In [3]:
# Calculate the values of coefficients

beta_results = beta_array[29:]
beta_vals = np.array([np.append(beta_array[r:r+29][::-1],1) for r in range(30)])
a_seq = np.linalg.inv(beta_vals) @ beta_results

gamma_results = gamma_array[29:]
gamma_vals = np.array([np.append(gamma_array[r:r+29][::-1],1) for r in range(30)])
b_seq = np.linalg.inv(gamma_vals) @ gamma_results

In [4]:
# Calculate and predict 50 R(t) with SIR model

reproduce_0 = beta_array / gamma_array
S_pred = S_array[-1]
X_pred = X_array[-1]
R_pred = R_array[-1]

reproduce_1 = np.zeros(50)
for r in range(50):
    beta_pred = np.dot(a_seq, np.append(beta_array[30+r:][::-1],1))
    beta_array = np.append(beta_array, beta_pred)
    S_pred = - beta_pred * S_pred * X_pred / n + S_pred

    gamma_pred = np.dot(b_seq, np.append(gamma_array[30+r:][::-1],1))
    gamma_array = np.append(gamma_array, gamma_pred)
    R_pred = gamma_pred * X_pred + R_pred

    X_pred = n - S_pred - R_pred

    reproduce_1[r] = beta_pred / gamma_pred

/var/folders/dz/lgsl3l_d7fv4z40yrx07rfz80000gn/T/ipykernel_70261/1663457792.py:12: RuntimeWarning: overflow encountered in scalar multiply
  S_pred = - beta_pred * S_pred * X_pred / n + S_pred
/var/folders/dz/lgsl3l_d7fv4z40yrx07rfz80000gn/T/ipykernel_70261/1663457792.py:12: RuntimeWarning: invalid value encountered in scalar add
  S_pred = - beta_pred * S_pred * X_pred / n + S_pred


In [5]:
# Calculate the values of I(t)
I_array = -np.diff(S_array)

# solve the values of w(s)
w_array = np.zeros(58)
for r in range(1,59):
    w_array[r-1] =(I_array[r] / reproduce_0[r] - np.dot(I_array[r-1:0:-1], w_array[:r-1])) / I_array[0]

# predict 50 R(t) by w(s) and TSI model


In [6]:
reproduce_1

array([-9.68603810e+01,  8.63391350e+02,  2.95953345e+02, -1.15568659e+02,
        2.19306745e+02,  2.40784199e+03, -8.69138849e+02, -3.29949402e+02,
       -1.79021531e+02,  7.95135662e+02, -1.96726729e+03,  5.61757736e+04,
       -3.59966516e+02,  2.36002369e+03, -1.27438652e+03,  1.93020356e+03,
       -5.51675985e+03,  4.37859268e+03, -2.03781201e+04, -2.84775260e+04,
        1.04421985e+05,  1.47108240e+04,  7.52301319e+04, -3.26645250e+04,
        3.54167839e+04, -6.20970774e+04,  1.14407768e+05, -1.64305333e+05,
        8.23120220e+04,  2.28930161e+05, -3.11092749e+05,  3.72099151e+05,
        1.70160790e+06,  1.80330242e+05, -8.92237793e+05,  4.39229651e+05,
       -7.44873110e+05,  2.24793890e+06, -1.32178629e+06,  3.89999285e+06,
        1.64696038e+07, -8.66411166e+06, -9.20452400e+06, -1.44779908e+07,
        1.14446089e+07, -1.00292641e+07,  1.52241235e+07, -2.72496996e+07,
        4.00041785e+07, -2.23768618e+07])

In [8]:
w_array

array([ 1.17596556e+02, -3.63049019e+02, -9.19375086e+02,  3.57601984e+03,
        2.30508365e+04, -8.99072317e+04, -3.08481323e+05,  1.43052582e+06,
        5.47294916e+06, -2.73272870e+07, -8.31211727e+07,  4.76605486e+08,
        1.34541310e+09, -8.59109885e+09, -2.07494476e+10,  1.51528650e+11,
        3.21816399e+11, -2.68180949e+12, -4.86074775e+12,  4.71054287e+13,
        7.22442942e+13, -8.25525310e+14, -1.04056879e+15,  1.44033237e+16,
        1.44124848e+16, -2.50488229e+17, -1.87236078e+17,  4.34043499e+18,
        2.17433370e+18, -7.49600891e+19, -1.93698937e+19,  1.29018710e+21,
        2.38787548e+19, -2.21326156e+22,  4.88045926e+21,  3.78408681e+23,
       -1.74137234e+23, -6.44812447e+24,  4.52758303e+24,  1.09503657e+26,
       -1.03819522e+26, -1.85318330e+27,  2.22342315e+27,  3.12508277e+28,
       -4.56007850e+28, -5.25055431e+29,  9.07544745e+29,  8.78783586e+30,
       -1.76647444e+31, -1.46490161e+32,  3.37962702e+32,  2.43154400e+33,
       -6.37722218e+33, -

In [6]:
beta_array / gamma_array

array([ 0.15984312,  1.05038569,  0.        ,  0.51869462,  0.68308344,
        0.        ,  1.30067201,  8.08168586,  0.78740157,  1.57944366,
        0.97207262,  4.72866472,  2.06198853,  1.0760659 ,  2.23147337,
       17.21763085,  2.41523456,  0.67563646,  1.87924134,  4.55729167,
        1.23703492,  0.74220683,  1.95421017,  4.50225113,  2.88076152,
        0.88183422,  2.12389381,  0.75987842, 10.64368981,  5.33130236,
        0.65397471,  4.20275089,  0.74862935,  0.50186407,  3.62127996,
        3.90398067,  9.39104303,  0.17308225,  0.33993523,  2.230276  ,
        0.58873003,  3.25842697,  0.35773376,  6.81237582,  1.19352089,
        3.41685649,  1.28939828,  1.79924032,  5.14488468,  6.36246148,
        1.03473762,  6.84030518,  0.71401371,  1.86219739,  4.66272925,
        3.74765771,  1.70454545,  3.3021464 ,  0.81196581])